# Домашнее задание 3
## Yes/No Questions

deadline: 3 декабря 2019, 23:59

В этом домашнем задании вы будете работать с корпусом BoolQ. Корпус состоит из вопросов, предполагающих бинарный ответ (да / нет), абзацев из Википедии,  содержащих ответ на вопрос, заголовка статьи, из которой извлечен абзац и непосредственно ответа (true / false).

Корпус описан в статье:

Christopher Clark, Kenton Lee, Ming-Wei Chang, Tom Kwiatkowski, Michael Collins, Kristina Toutanova
BoolQ: Exploring the Surprising Difficulty of Natural Yes/No Questions

https://arxiv.org/abs/1905.10044


Корпус (train-dev split) доступен в репозитории проекта:  https://github.com/google-research-datasets/boolean-questions

Используйте для обучения train часть корпуса, для валидации и тестирования – dev часть. 

Каждый бонус пункт оцениватся в 1 балл. 

### Пример вопроса: 
question: is batman and robin a sequel to batman forever

title: Batman & Robin (film)

answer: true

passage: With the box office success of Batman Forever in June 1995, Warner Bros. immediately commissioned a sequel. They hired director Joel Schumacher and writer Akiva Goldsman to reprise their duties the following August, and decided it was best to fast track production for a June 1997 target release date, which is a break from the usual 3-year gap between films. Schumacher wanted to homage both the broad camp style of the 1960s television series and the work of Dick Sprang. The storyline of Batman & Robin was conceived by Schumacher and Goldsman during pre-production on A Time to Kill. Portions of Mr. Freeze's back-story were based on the Batman: The Animated Series episode ''Heart of Ice'', written by Paul Dini.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание сдается через anytask, инвайты будут дополнительно высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import jsonlines
from unidecode import unidecode
import os
import pickle
import spacy

DATA_PREF = 'data/'
os.makedirs(DATA_PREF, exist_ok=True)
TRAIN_FILE_PATH = DATA_PREF + 'train.jsonl'
DEV_FILE_PATH = DATA_PREF + 'dev.jsonl'
FAST_TEXT_PREF = 'FastText/'
os.makedirs(FAST_TEXT_PREF, exist_ok=True)
FAST_TEXT_TRAIN_FILE_PATH = FAST_TEXT_PREF + 'train.txt'
FAST_TEXT_DEV_FILE_PATH = FAST_TEXT_PREF + 'dev.txt'
FAST_TEXT_MODEL_NAME = FAST_TEXT_PREF + 'model.pickle'

In [2]:
def open_dataset(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        df = pd.DataFrame.from_records(list(reader))
    df['passage'] = df['passage'].map(unidecode)
    df['question'] = df['question'].map(unidecode)
    df['title'] = df['title'].map(unidecode)
    return df
    
df_train = open_dataset(TRAIN_FILE_PATH)    
df_dev = open_dataset(DEV_FILE_PATH)

## Часть 1. [1 балл] Эксплоративный анализ
1. Посчитайте долю yes и no классов в корпусе
2. Оцените среднюю длину вопроса
3. Оцените среднюю длину параграфа
4. Предположите, по каким эвристикам были собраны вопросы (или найдите ответ в статье). Продемонстриуйте, как эти эвристики повлияли на структуру корпуса. 

In [3]:
df_train[:5]

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/'pe:rZ@n, -S@n/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,Powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,False,As with other games in The Elder Scrolls serie...


In [4]:
print('Доля True меток:\t\t\t\t', (df_train['answer'].values == True).mean())
print('Доля False меток:\t\t\t\t', (df_train['answer'].values == False).mean())
len_func = np.vectorize(lambda x: len(x))
print('Средняя длина вопроса:\t\t\t\t', len_func(df_train['question'].values).mean(), 'символов')
print('Средняя длина параграфа:\t\t\t', len_func(df_train['passage'].values).mean(), 'символов')
yes_no_words_set = set(['did', 'do', 'does', 'is', 'are', 'was', 'were', 'have', 'has', 'can', 'could', 'will', 'would'])
is_in_set_func = np.vectorize(lambda x: x.split()[0].lower() in yes_no_words_set)
print('Доля вопросов, начинающихся с ключевых слов: \t', is_in_set_func(df_train['question'].values).mean())

Доля True меток:				 0.6231038506417736
Доля False меток:				 0.37689614935822635
Средняя длина вопроса:				 43.99204412856688 символов
Средняя длина параграфа:			 565.7658852232948 символов
Доля вопросов, начинающихся с ключевых слов: 	 0.9570382942611647


Вопрос, задаваемый гуглу, классифицировался как "yes/no question", если он начинался с ключевого слова. (слова можно посмотреть в ячейке выше). Соответственно, 95% вопросов датасета начинаются именно с них

## Часть 2. [1 балл] Baseline
1. Оцените accuracy точность совсем простого базового решения: присвоить каждой паре вопрос-ответ в dev части самый частый класс из train части
2. Оцените accuracy чуть более сложного базового решения: fasttext на текстах, состоящих из склееных вопросов и абзацев (' '.join([question, passage]))

Почему fasttext плохо справляется с этой задачей?

In [5]:
y_true = df_dev['answer'].values
y_pred = np.ones_like(y_true)
print('Точность при выставлении всем меткам True: %f' % (y_true == y_pred).mean())

Точность при выставлении всем меткам True: 0.621713


In [6]:
import fasttext
from tqdm import tqdm_notebook as tqdm

def write_to_fasttext_dataset(df, res_file_path):
    f = open(res_file_path, 'w')
    for i in tqdm(range(df.shape[0])):
        obj = df.iloc[i]
        line = '__label__' + str(int(obj['answer'])) + ' '
        line = line + obj['passage'] + ' ' + obj['question'] + '\n'
        f.write(line)
    f.close()

In [7]:
write_to_fasttext_dataset(df_train, FAST_TEXT_TRAIN_FILE_PATH)
write_to_fasttext_dataset(df_dev, FAST_TEXT_DEV_FILE_PATH)

C:\Users\NMikhaylov\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [8]:
def train_fasttext_model():
    if os.path.exists(FAST_TEXT_MODEL_NAME) and os.path.isfile(FAST_TEXT_MODEL_NAME):
        print('Found backup')
        return fasttext.load_model(FAST_TEXT_MODEL_NAME)
    model = fasttext.train_supervised(FAST_TEXT_TRAIN_FILE_PATH, label_prefix='__label__', epoch=400)
    model.save_model(FAST_TEXT_MODEL_NAME)
    return model

fasttext_model = train_fasttext_model()

Found backup


In [9]:
def make_input_fasttext(df):
    inp = []
    for i in tqdm(range(df.shape[0])):
        obj = df.iloc[i]
        line = obj['passage'] + ' ' + obj['question']
        inp.append(line)
    return inp
dev_fasttext_input = make_input_fasttext(df_dev)
fasttext_prediction = fasttext_model.predict(dev_fasttext_input)[0]
fasttext_prediction = np.array([int(s[0][len('__label__'):]) for s in fasttext_prediction])

C:\Users\NMikhaylov\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
print((fasttext_prediction == y_true).mean())

0.6513761467889908


FastText плохо справился, потому что классы несбалансированны

## Часть 3. [1 балл] Используем эмбеддинги предложений
1. Постройте BERT эмбеддинги вопроса и абзаца. Обучите логистическую регрессию на конкатенированных эмбеддингах вопроса и абзаца и оцените accuracy этого решения. 

[bonus] Используйте другие модели эмбеддингов, доступные, например, в библиотеке 🤗 Transformers. Какая модель эмбеддингов даст лучшие результаты?

[bonus] Предложите метод аугментации данных и продемонстрируйте его эффективность. 

In [11]:
from transformers import *
MODELS = [(BertModel,       BertTokenizer,       'bert-base-uncased'),
          (RobertaModel,    RobertaTokenizer,    'roberta-base')]

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [12]:
def sentence2vector(sentence, tokenizer):
    tokenized_sent = tokenizer.encode(sentence, add_special_tokens=True, max_length=256)
    return torch.LongTensor(tokenized_sent)

def sentences2tensor(sentences, tokenizer):
    vectors = [sentence2vector(sent, tokenizer) for sent in sentences]
    lens = [len(vec) for vec in vectors]
    max_len = max(lens)
    
    matrix = torch.zeros((len(vectors), max_len), dtype=torch.long)
    mask = torch.zeros_like(matrix)
    
    for i, cur_len in enumerate(lens):
        matrix[i, :cur_len] = vectors[i]
        mask[i, :cur_len] = 1.
    
    return matrix, mask

In [13]:
#sentence2vector(df_train['question'][0], BertTokenizer.from_pretrained(MODELS[0][2]), 10)
sentences2tensor(df_train['question'][0:2], BertTokenizer.from_pretrained(MODELS[0][2]))


(tensor([[ 101, 2079, 4238, 1998, 7041, 3713, 1996, 2168, 2653,  102,    0,    0,
             0,    0,    0],
         [ 101, 2079, 2204, 3520, 8486, 5794, 4277, 4047, 2216, 2040, 2393, 2012,
          2019, 4926,  102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
batch_size=256
bert_emb_size = 768

def calc_embeddings(df, model, tokenizer):
    with torch.no_grad():
        question_embeddings = torch.zeros((df.shape[0], bert_emb_size))
        passage_embeddings = torch.zeros((df.shape[0], bert_emb_size))
        bar = tqdm(total=df.shape[0])
        for begin in range(0, df.shape[0], batch_size):
            end = begin + batch_size
            if end >= df.shape[0]:
                end = df.shape[0]
            tokenized, mask = sentences2tensor(df['question'].values[begin:end], tokenizer)
            batch_embedding, _ = model(tokenized.cuda(), mask.cuda())
            #print(batch_embedding.shape)
            batch_embedding = torch.mean(batch_embedding, dim=1)
            #print(batch_embedding.shape)
            question_embeddings[begin:end] = batch_embedding

            tokenized, mask = sentences2tensor(df['passage'].values[begin:end], tokenizer)
            batch_embedding, _ = model(tokenized.cuda(), mask.cuda())
            batch_embedding = torch.mean(batch_embedding, dim=1)
            passage_embeddings[begin:end] = batch_embedding
            bar.update(end - begin)
    
    embed = torch.cat((question_embeddings, passage_embeddings), dim=-1)
    #print(embed.shape)
    return embed
        

for model_class, tokenizer_class, pretrained_weights in MODELS:
    print(pretrained_weights)
    
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    model = model.cuda()
    model = model.eval()
    
    x_train = calc_embeddings(df_train, model, tokenizer)
    y_train = df_train['answer']
    x_test = calc_embeddings(df_dev, model, tokenizer)
    y_test = df_dev['answer']
    log_reg = LogisticRegression().fit(x_train, y_train)
    y_pred = log_reg.predict(x_test)
    print(classification_report(y_test, y_pred))
    print('--------------------------------------------------------------------')
    model = model.cpu()
    del model
    gc.collect()
    torch.cuda.empty_cache()

bert-base-uncased


              precision    recall  f1-score   support

       False       0.55      0.45      0.50      1237
        True       0.70      0.77      0.73      2033

    accuracy                           0.65      3270
   macro avg       0.62      0.61      0.62      3270
weighted avg       0.64      0.65      0.64      3270

--------------------------------------------------------------------
roberta-base


              precision    recall  f1-score   support

       False       0.58      0.41      0.48      1237
        True       0.70      0.82      0.75      2033

    accuracy                           0.67      3270
   macro avg       0.64      0.62      0.62      3270
weighted avg       0.65      0.67      0.65      3270

--------------------------------------------------------------------


### Итого мы попробовали классикечкий BERT и roBERTa. Как видим по F-мере получилось, что roBERTa лучше справлвяется с задачей, но всего лишь на 0.02 и то если выставить всем меткам True, то будет лучше

## Часть 3. [3 балла] DrQA-подобная архитектура

Основана на статье: Reading Wikipedia to Answer Open-Domain Questions

Danqi Chen, Adam Fisch, Jason Weston, Antoine Bordes

https://arxiv.org/abs/1704.00051

Архитектура DrQA предложена для задачи SQuAD, но легко может быть адаптирована к текущему заданию. Модель состоит из следующих блоков:
1. Кодировщик абзаца [paragraph encoding] – LSTM, получаящая на вход вектора слов, состоящие из: 
* эмбеддинга слова (w2v или fasttext)
* дополнительных признаков-индикаторов, кодирующих в виде one-hot векторов часть речи слова, является ли оно именованной сущностью или нет, встречается ли слово в вопросе или нет 
* выровненного эмбеддинга вопроса, получаемого с использованием soft attention между эмбеддингами слов из абзаца и эмбеддингом вопроса.

$f_{align}(p_i) = \sum_j􏰂 a_{i,j} E(q_j)$, где $E(q_j)$ – эмбеддинг слова из вопроса. Формула для $a_{i,j}$ приведена в статье. 

2. Кодировщик вопроса [question encoding] – LSTM, получаящая на вход эмбеддинги слов из вопроса. Выход кодировщика: $q = 􏰂\sum_j􏰂  b_j q_j$. Формула для $b_{j}$ приведена в статье. 

3. Слой предсказания. 

Предложите, как можно было модифицировать последний слой предсказания в архитектуре DrQA, с учетом того, что итоговое предсказание – это метка yes / no, предсказание которой проще, чем предсказание спана ответа для SQuAD.

Оцените качество этой модели для решения задачи. 

[bonus] Замените входные эмбеддинги и все дополнительные признаки, используемые кодировщиками, на BERT эмбеддинги. Улучшит ли это качество результатов?

In [17]:
def split_to_sentance(df):
    df['passage'] = df['passage'].map(unidecode)
    df['question'] = df['question'].map(unidecode)
    df['title'] = df['title'].map(unidecode)

## Часть 4. [3 балла] BiDAF-подобная архитектура

Основана на статье: Bidirectional Attention Flow for Machine Comprehension

Minjoon Seo, Aniruddha Kembhavi, Ali Farhadi, Hannaneh Hajishirzi

https://arxiv.org/abs/1611.01603

Архитектура BiDAF предложена для задачи SQuAD, но легко может быть адаптирована к текущему заданию. Модель состоит из следующих блоков:
1. Кодировщик  получает на вход два представления слова: эмбеддинг слова и полученное из CNN посимвольное представление слова. Кодировщики для вопроса и для параграфа одинаковы. 
2. Слой внимания (детальное описание приведено в статье, см. пункт Attention Flow Layer)
3. Промежуточный слой, который получает на вход контекстуализированные эмбеддинги слов из параграфа, состоящие из трех частей (выход кодировщика параграфа,   Query2Context (один вектор) и Context2Query (матрица) выравнивания

4. Слой предсказания. 

Предложите, как можно было модифицировать последний слой предсказания в архитектуре BiDAF, с учетом того, что итоговое предсказание – это метка yes / no, предсказание которой проще, чем предсказание спана ответа для SQuAD.

Оцените качество этой модели для решения задачи. 

[bonus] Замените входные эмбеддинги и все дополнительные признаки, используемые кодировщиками, на BERT эмбеддинги. Улучшит ли это качество результатов?

Сравнение DrQA и BiDAF:
    
![](https://www.researchgate.net/profile/Felix_Wu6/publication/321069852/figure/fig1/AS:560800147881984@1510716582560/Schematic-layouts-of-the-BiDAF-left-and-DrQA-right-architectures-We-propose-to.png)

## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?